In [86]:
import numpy as np
import os
import math
from anytree import Node, RenderTree, PreOrderIter, AsciiStyle
import time

Lire le contenue de tous les fichiers du dossier textes

Lire le contenu du texte (entrer le numero du texte)

In [87]:
# text_number = input("Quel texte voulez-vous lire ? (entrez un num entre 1 et 5) : ")

# file_name = f"textes/texte_{text_number}.txt"
# Message = open(file_name, "r").read()

# print(Message)

all_files = os.listdir("textes")
text = [file for file in all_files if file.endswith(".txt")]

messages = {}
result = {}

for file in text:
    with open("textes/" + file, "r") as f:
        messages[file] = f.read()
        
print(result)
print(messages)

{}
{'texte_1.txt': 'AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA', 'texte_2.txt': 'AAAAABBBBBBBBBAAAAAAAAABBBBBBBCBBBBAAACACBCBBBBBBBBBAAAAAAAAACAAAABBBBAAAAABB', 'texte_3.txt': 'FOFOFOEOKOWKOOWONOOOOOOWOENFOEOOOJSODNOJFSOJKOOOOBV', 'texte_4.txt': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla posuere ligula id nibh egestas, id sollicitudin nisi interdum. Integer a elit sed sem egestas interdum vitae id risus. ', 'texte_5.txt': 'Aliquam placerat dictum lectus vel pellentesque. Vestibulum semper dui lacus, sed commodo dolor maximus a. Praesent convallis in quam in posuere. Ut eleifend tortor a laoreet imperdiet. Sed pharetra risus sit amet velit cursus semper. Nam congue ac lectus sit amet laoreet. Integer quis lacinia turpis. Maecenas erat orci, porttitor ac ligula dapibus, interdum fringilla nunc. Proin auctor, arcu ac fermentum ultricies, erat justo laoreet nunc, sit amet lacinia lorem elit vel dolor. Aenean fermentum, mi sed i

Convertir le contenu du texte en binaire afin de pouvoir appliquer la methode de distance entre 1 et 0

In [88]:
for file in text:
    Message = messages[file]
    binary_sequence = ''.join(format(ord(i), '08b') for i in Message)
    arbresymb = [[Message[0], Message.count(Message[0]), Node(Message[0])]]
    nbsymboles = 1
    # rOriginale = np.ceil(np.log2(nbsymboles))*len(Message)
    result[file] = {
        "message": Message,
        "length": len(Message),
        "binary_sequence": binary_sequence,
        "arbresymb": arbresymb,
        "nbsymboles": nbsymboles
        # "longueurOriginale": longueurOriginale
    }

    for file, data in result.items():
        print(f"File: {file}")
        for key, value in data.items():
            print(f"{key}: {value}")
        print()

File: texte_1.txt
message: AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA
length: 84
binary_sequence: 010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000010010000100100000101000010010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000100100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000100100001001000001010000010100000101000001010000010100000101000001010000010100001001000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001
arbresymb: [['A', 77, Node('/A')]]
nbsymboles: 1

File: texte_1.txt
message: AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA
length: 84
binary_sequenc

Calculer la distance entre les 0 et les 1 de facon qu'on elimine les 1 et ne laisse que les distances. Exemple : 000100110 --> 3201

In [89]:
for file in result:
    print(f"File: {file}")
    Message = messages[file]
    binary_sequence = ''.join(format(ord(i), '08b') for i in Message)
    binary_sequence = '1' + binary_sequence + '1'
    distances = []
    count = 0

    for i in range(len(binary_sequence)):
        if binary_sequence[i] == '0':
            count += 1
        else:
            distances.append(count)
            count = 0

    distances = distances[1:-1]
    distances_str = ''.join(str(distance) for distance in distances)
    Message = distances_str
    print(distances_str)

File: texte_1.txt
151515151515151515151515151515151514242514251515151515151515151515142515151515151515151515151515142425151515151515151425151515151515151515151515151515151515151515151515
File: texte_2.txt
151515151514242424242424242425151515151515151514242424242424240142424242515151401514014240142424242424242424251515151515151515140151515151424242425151515151424
File: texte_3.txt
13022000130220001302200013112000121012000111001210120001200011100120001200220001200012000120001200012000111001200013112002302200013112000120001200012121201200013320022000121230212012000121221012000120001200012000142110
File: texte_4.txt
12030100010022021101012601210050020100111010126023010001010301000100236002010121001460410101102110014104603010100010100200201021103010013021100130011100236041023012100501210020103010121010020200260211010301210014100362002001110103010304260050100010020100111021100220212601030121020010011101030426012102460100201210320156021102001021100201001304100202104601210246002010100010103010

Préparation pour la création de l'arbre. On trouve les feuilles (symboles) et leurs poids (nb occurences).

In [90]:
for file in result:
    print(f"File: {file}")
    Message = messages[file]
    #Liste qui sera modifié jusqu'à ce qu'elle contienne seulement la racine de l'arbre
    ArbreSymb = result[file]["arbresymb"]
    # ArbreSymb =[[Message[0], Message.count(Message[0]), Node(Message[0])]]
    #dictionnaire obtenu à partir de l'arbre.
    dictionnaire = [[Message[0], '']]
    nbsymboles = result[file]["nbsymboles"]

    #Recherche des feuilles de l'arbre
    for i in range(1,len(Message)):
        if not list(filter(lambda x: x[0] == Message[i], arbresymb)):
            arbresymb += [[Message[i], Message.count(Message[i]),Node(Message[i])]]
            dictionnaire += [[Message[i], '']]
            nbsymboles += 1

    longueurOriginale = np.ceil(np.log2(nbsymboles))*len(Message)
    print('Longueur originale :', longueurOriginale)

    OccSymb = arbresymb.copy()

File: texte_1.txt
Longueur originale : 84.0
File: texte_2.txt
Longueur originale : 77.0
File: texte_3.txt
Longueur originale : 204.0
File: texte_4.txt
Longueur originale : 910.0
File: texte_5.txt
Longueur originale : 5332.0


Affichage des feuilles trouvées. Les feuilles sont triées pour la construction de l'arbre selon l'algorithme Huffman

In [91]:
for file in result:
    print(f"File: {file}")
    ArbreSymb = result[file]["arbresymb"]
    ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])
    print(ArbreSymb)

File: texte_1.txt
[['A', 77, Node('/A')]]
File: texte_2.txt
[['A', 36, Node('/A')]]
File: texte_3.txt
[['F', 5, Node('/F')]]
File: texte_4.txt
[['L', 1, Node('/L')]]
File: texte_5.txt
[['D', 1, Node('/D')], ['V', 1, Node('/V')], ['b', 1, Node('/b')], ['h', 1, Node('/h')], ['I', 1, Node('/I')], ['v', 1, Node('/v')], ['j', 1, Node('/j')], ['Q', 1, Node('/Q')], ['A', 2, Node('/A')], ['S', 2, Node('/S')], [',', 2, Node('/,')], ['U', 2, Node('/U')], ['M', 2, Node('/M')], ['E', 3, Node('/E')], ['K', 3, Node('/K')], ['W', 3, Node('/W')], ['N', 3, Node('/N')], ['J', 3, Node('/J')], ['p', 3, Node('/p')], ['.', 3, Node('/.')], ['x', 3, Node('/x')], ['P', 3, Node('/P')], ['c', 4, Node('/c')], ['C', 5, Node('/C')], ['F', 5, Node('/F')], ['g', 5, Node('/g')], ['o', 6, Node('/o')], ['m', 6, Node('/m')], ['B', 7, Node('/B')], ['r', 8, Node('/r')], ['a', 8, Node('/a')], ['n', 8, Node('/n')], ['q', 8, Node('/q')], ['u', 9, Node('/u')], ['d', 9, Node('/d')], ['l', 9, Node('/l')], ['t', 13, Node('/t')], 

In [92]:
for file in result:
    
    ArbreSymb = result[file]["arbresymb"]
    while len(ArbreSymb) > 1:
        print(f"File: {file}")
        #Fusion des noeuds de poids plus faibles
        symbfusionnes = ArbreSymb[0][0] + ArbreSymb[1][0]
        #Création d'un nouveau noeud
        noeud = Node(symbfusionnes)
        temp = [symbfusionnes, ArbreSymb[0][1] + ArbreSymb[1][1], noeud]
        #Ajustement de l'arbre pour connecter le nouveau avec ses parents
        ArbreSymb[0][2].parent = noeud
        ArbreSymb[1][2].parent = noeud
        #Enlève les noeuds fusionnés de la liste de noeud à fusionner.
        del ArbreSymb[0:2]
        #Ajout du nouveau noeud à la liste et tri.
        ArbreSymb += [temp]
        #Pour affichage de l'arbre ou des sous-branches
        print('\nArbre actuel:\n\n')
        for i in range(len(ArbreSymb)):
            if len(ArbreSymb[i][0]) > 1:
                print(RenderTree(ArbreSymb[i][2], style=AsciiStyle()).by_attr())
        ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])
        print(ArbreSymb)

File: texte_5.txt

Arbre actuel:


AB
|-- A
+-- B
[['D', 1, Node('/D')], ['V', 1, Node('/V')], ['b', 1, Node('/b')], ['h', 1, Node('/h')], ['I', 1, Node('/I')], ['v', 1, Node('/v')], ['j', 1, Node('/j')], ['Q', 1, Node('/Q')], ['S', 2, Node('/S')], [',', 2, Node('/,')], ['U', 2, Node('/U')], ['M', 2, Node('/M')], ['E', 3, Node('/E')], ['K', 3, Node('/K')], ['W', 3, Node('/W')], ['N', 3, Node('/N')], ['J', 3, Node('/J')], ['p', 3, Node('/p')], ['.', 3, Node('/.')], ['x', 3, Node('/x')], ['P', 3, Node('/P')], ['c', 4, Node('/c')], ['C', 5, Node('/C')], ['F', 5, Node('/F')], ['g', 5, Node('/g')], ['o', 6, Node('/o')], ['m', 6, Node('/m')], ['r', 8, Node('/r')], ['a', 8, Node('/a')], ['n', 8, Node('/n')], ['q', 8, Node('/q')], ['u', 9, Node('/u')], ['d', 9, Node('/d')], ['l', 9, Node('/l')], ['AB', 9, Node('/AB')], ['t', 13, Node('/t')], ['s', 15, Node('/s')], ['f', 15, Node('/f')], ['e', 19, Node('/e')], ['i', 21, Node('/i')], ['O', 26, Node('/O')], [' ', 28, Node('/ ')]]
File: texte_5.tx

On traverse l'arbre des symboles par parcours préfix, et on construit un arbre semblable de codes binaires. Le premier enfant avec 0, le deuxième avec 1. Ensuite, on utilisera les 2 arbres pour construire le dictionnaire.

In [93]:
ArbreCodes = Node('')
noeud = ArbreCodes
#print([node.name for node in PreOrderIter(ArbreSymb[0][2])])
parcoursprefix = [node for node in PreOrderIter(ArbreSymb[0][2])]
parcoursprefix = parcoursprefix[1:len(parcoursprefix)] #ignore la racine

Prevdepth = 0 #pour suivre les mouvements en profondeur dans l'arbre
for node in parcoursprefix:  #Liste des noeuds
    if Prevdepth < node.depth: #On va plus profond dans l'arbre, on met un 0
        temp = Node(noeud.name + '0')
        noeud.children = [temp]
        if node.children: #On avance le "pointeur" noeud si le noeud ajouté a des enfants.
            noeud = temp
    elif Prevdepth == node.depth: #Même profondeur, autre feuille, on met un 1
        temp = Node(noeud.name + '1')
        noeud.children = [noeud.children[0], temp]  #Ajoute le deuxième enfant
        if node.children: #On avance le "pointeur" noeud si le noeud ajouté a des enfants.
            noeud = temp
    else:
        for i in range(Prevdepth-node.depth): #On prend une autre branche, donc on met un 1
            noeud = noeud.parent #On remontre dans l'arbre pour prendre la prochaine branche non explorée.
        temp = Node(noeud.name + '1')
        noeud.children = [noeud.children[0], temp]
        if node.children:
            noeud = temp

    Prevdepth = node.depth

print('\nArbre des codes:\n\n',RenderTree(ArbreCodes, style=AsciiStyle()).by_attr())
print('\nArbre des symboles:\n\n', RenderTree(ArbreSymb[0][2], style=AsciiStyle()).by_attr())


Arbre des codes:

 
|-- 0
|   |-- 00
|   |   |-- 000
|   |   |   |-- 0000
|   |   |   +-- 0001
|   |   |       |-- 00010
|   |   |       |   |-- 000100
|   |   |       |   +-- 000101
|   |   |       +-- 00011
|   |   |           |-- 000110
|   |   |           +-- 000111
|   |   +-- 001
|   +-- 01
|       |-- 010
|       |   |-- 0100
|       |   +-- 0101
|       +-- 011
|           |-- 0110
|           |   |-- 01100
|           |   +-- 01101
|           +-- 0111
|               |-- 01110
|               +-- 01111
+-- 1
    |-- 10
    |   |-- 100
    |   |   |-- 1000
    |   |   |   |-- 10000
    |   |   |   |   |-- 100000
    |   |   |   |   |   |-- 1000000
    |   |   |   |   |   +-- 1000001
    |   |   |   |   +-- 100001
    |   |   |   |       |-- 1000010
    |   |   |   |       +-- 1000011
    |   |   |   +-- 10001
    |   |   |       |-- 100010
    |   |   |       |   |-- 1000100
    |   |   |       |   |   |-- 10001000
    |   |   |       |   |   +-- 10001001
    |   |   |       

In [94]:
for file in result:
    ArbreSymb = result[file]["arbresymb"]
    ArbreSymbList = [node for node in PreOrderIter(ArbreSymb[0][2])]
    ArbreCodeList = [node for node in PreOrderIter(ArbreCodes)]

    for i in range(len(ArbreSymbList)):
        if ArbreSymbList[i].is_leaf: #Génère des codes pour les feuilles seulement
            temp = list(filter(lambda x: x[0] == ArbreSymbList[i].name, dictionnaire))
            if temp:
                indice = dictionnaire.index(temp[0])
                dictionnaire[indice][1] = ArbreCodeList[i].name
    result[file]["dictionnaire"] = dictionnaire

for file, data in result.items():
    print(f"File: {file}")
    print(result[file]["dictionnaire"])

File: texte_1.txt
[['A', ''], ['q', ''], ['x', ''], ['P', ''], ['U', ''], ['f', ''], ['M', ''], ['j', ''], ['Q', '']]
File: texte_2.txt
[['A', ''], ['q', ''], ['x', ''], ['P', ''], ['U', ''], ['f', ''], ['M', ''], ['j', ''], ['Q', '']]
File: texte_3.txt
[['A', ''], ['q', ''], ['x', ''], ['P', ''], ['U', ''], ['f', ''], ['M', ''], ['j', ''], ['Q', '']]
File: texte_4.txt
[['A', ''], ['q', ''], ['x', ''], ['P', ''], ['U', ''], ['f', ''], ['M', ''], ['j', ''], ['Q', '']]
File: texte_5.txt
[['A', ''], ['q', ''], ['x', ''], ['P', ''], ['U', ''], ['f', ''], ['M', ''], ['j', ''], ['Q', '']]


Codage du message avec le dictionnaire

In [95]:
for file, data in result.items():
    MessageCode = []
    longueur = 0
    tempsDebut = time.time()
    for i in range(len(Message)):
        substitution = list(filter(lambda x: x[0] == Message[i], dictionnaire))
        MessageCode += [substitution[0][1]]
        longueur += len(substitution[0][1])
    tempsFin = time.time()
    result[file]["MessageCode"] = MessageCode
    result[file]["longueur"] = longueur
    result[file]["temps"] = round(tempsFin - tempsDebut, 5)
    print(MessageCode)

IndexError: list index out of range

Longueur en bits du message codé et celle de l'original

In [ ]:
for file, data in result.items():
    longueur = result[file]["longueur"]
    longueurOriginale = result[file]["longueurOriginale"]
    ratio = result[file]["ratio"]
    print("Longueur = {0}".format(longueur))
    print("Longueur originale = {0}".format(longueurOriginale))
    print("Ratio = {0}".format(longueur/longueurOriginale))

KeyError: 'longueur'

Espérance de la longueur d'un code avec Huffman vs Entropie

In [ ]:
print('Espérance: ' + str(longueur/len(Message)))
entropie =0
for i in range(nbsymboles):
  entropie = entropie-(OccSymb[i][1]/len(Message))*math.log(OccSymb[i][1]/len(Message),2)

print('Entropie: ' + str(entropie))


Espérance: 4.201800450112528
Entropie: 338.56721035710007
